### Agentic RAG

Here is the workflow for current implementation

- Use two vector db's 
    - first db will have LangGraph related blogs
    - second db will have LangChain related blogs
- An Agent (Agent 1) will decide based on the user query to choose first db or second db
- Once the related documents are fetched, we check if the documents have correct information
    - If the information is selected document context is not appropriate
    - Rewrite (Agent 2) the query and send the updated query back to the agent
- If the info is correct - pass it as contect to the Generator (Agent 3) and finally output the response
- If an unrelated query is asked agent (Agent 1) will directly reject and appropriate answer will be sent as output


In [ ]:
from langchain.chat_models import init_chat_model
from langchain.embeddings import init_embeddings
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, AnyMessage
from langchain_core.tools import tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.tools import create_retriever_tool 

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Send
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver

from typing import Annotated, TypedDict, Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import operator
import requests
import os

load_dotenv()

### Model and Embedding initialization

In [ ]:
model = init_chat_model(
    model="qwen/qwen3-32b",  # or any other Groq model
    model_provider="groq",
    temperature=0.7,
)

embeddings = init_embeddings("openai:text-embedding-3-small")

### Load, extract and flatten documents from Web

In [ ]:
#  Load and extract docs from LangGraph docs

urls_lg = [
    "https://docs.langchain.com/oss/python/langgraph/quickstart",
    "https://docs.langchain.com/oss/python/langgraph/workflows-agents",
    "https://docs.langchain.com/oss/python/langgraph/use-graph-api#map-reduce-and-the-send-api"
]

# load the docs from the urls 
docs_lg = [WebBaseLoader(url).load() for url in urls_lg]
docs_lg

# extract the inner list from docs and flatten it 
flat_doc_list_lg = [doc for sublist in docs_lg for doc in sublist]
flat_doc_list_lg

In [ ]:
#  Load and extract docs from LangGraph docs

urls_lc = [
    "https://docs.langchain.com/oss/python/langchain/overview",
    "https://docs.langchain.com/oss/python/langchain/agents",
    "https://docs.langchain.com/oss/python/langchain/guardrails"
]

# load the docs from the urls 
docs_lc = [WebBaseLoader(url).load() for url in urls_lc]
docs_lc

# extract the inner list from docs and flatten it 
flat_docs_list_lc = [doc for sublist in docs_lc for doc in sublist]
flat_docs_list_lc

### Text Splitter, Document Store (Vector Store) and Retriever (Setup)

In [ ]:
# Set up text splitter - recursive character based with chunk size of 1000 and chunk overlap of 100
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [ ]:
doc_chunks_lg = text_splitter.split_documents(flat_doc_list_lg)
doc_chunks_lg

vectorstore_faiss = FAISS.from_documents(doc_chunks_lg, embeddings)
retriever_lg = vectorstore_faiss.as_retriever()

print(retriever_lg.invoke("How to build agents?"))

In [ ]:
doc_lc_chunks = text_splitter.split_documents(flat_docs_list_lc)
doc_lc_chunks

vectorstore_chroma = Chroma.from_documents(doc_lc_chunks, embeddings)
retriever_lc = vectorstore_chroma.as_retriever()

print(retriever_lc.invoke("How to build chains?"))

### Setup Retriever as a tool

In [ ]:
retriever_langgraph_tool = create_retriever_tool(
    retriever=retriever_lg,
    name="retriever_lg_vector_db_blog",
    description="Search and extract information about LangGraph",
)

retriever_langchain_tool = create_retriever_tool(
    retriever=retriever_lc,
    name="retriever_lc_vector_db_blog",
    description="Search and extract information about LangChain",
)

### Status Till Now:

- Extracted documents from web about LangGraph
- It has information about LangGraph Implementation
- Used FAISS vectorstore to save the document and chunking them
- Created vectorstore as retriever
- Created retriever tool


### Combine tools

In [ ]:
tools = [retriever_langgraph_tool, retriever_langchain_tool]

### Bind the Model with tools

In [ ]:
model_with_tools = model.bind_tools(tools=tools)

### Build Graph

Step 1: Create an Agent that will use model with tools to get relavent documents based on query

In [ ]:
class AgentState(TypedDict):
    # add_messages is a reducer function, that defines how to add messages to the list
    # by default state would replace the entry, but with add_messages it would append to the list
    messages: Annotated[list[AnyMessage], add_messages]

In [ ]:
def agent(state: AgentState) -> AgentState:
    """ 
    Invokes the agent model to generate a response based on current state. Given the
    question, it will decide to call the retriever tool or end the call"""

    messages = state["messages"]
    response = model_with_tools.invoke({"messages": messages})
    return {"messages": [response]}

### Next Steps